In [1]:
from lsst.ts import salobj
from lsst.ts.idl.enums import MTMount
import logging
import asyncio
import os
import yaml

import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd
import astropy.units as u

from lsst_efd_client import EfdClient

# Before the tests:

In [2]:
script = salobj.Controller("Script", index=42658885)

In [3]:
mtm_sim = salobj.Remote(script.domain, "MTMount")
await mtm_sim.start_task

In [4]:
# this is to ensure that MTMount is connected and telemetry is send
mtm_sim_connected = await mtm_sim.evt_connected.aget(timeout=10.) # This is how you get an event/telemetry with await remote.type_name.aget() type being [evt,tel] and name being the name of the topic
mtm_commandable = mtm_sim_connected.command

if not mtm_sim_connected: #or not mtm_sim_telemetry_working:
    raise Exception("Mount not connected") #or telemetry not being received.

TimeoutError: 

In [5]:
#This is to check what the controller state is:
controller_state = await mtm_sim.evt_summaryState.aget(timeout=10.)
state = controller_state.summaryState
print(state)

1


In [6]:
#Print the available MTMount events, telemetry and commands
print("Telemery:", mtm_sim.salinfo.telemetry_names)
print("Events:",mtm_sim.salinfo.event_names)
print("Commands:",mtm_sim.salinfo.command_names)

Telemery: ('azimuth', 'azimuthDrives', 'cameraCableWrap', 'elevation', 'elevationDrives')
Events: ('appliedSettingsMatchStart', 'authList', 'axesInPosition', 'axesState', 'cameraCableWrapState', 'connected', 'error', 'errorCode', 'heartbeat', 'logLevel', 'logMessage', 'settingVersions', 'settingsApplied', 'simulationMode', 'softwareVersions', 'summaryState', 'target', 'warning')
Commands: ('abort', 'clearError', 'closeMirrorCovers', 'disable', 'disableCameraCableWrapTracking', 'enable', 'enableCameraCableWrapTracking', 'enterControl', 'exitControl', 'moveToTarget', 'openMirrorCovers', 'setAuthList', 'setLogLevel', 'setValue', 'standby', 'start', 'startTracking', 'stop', 'stopTracking', 'trackTarget')


In [7]:
#Send the MTMount simulator component into STANDBY
await salobj.set_summary_state(mtm_sim, salobj.State.STANDBY)

[<State.DISABLED: 1>, <State.STANDBY: 5>]

In [8]:
#Check if MTMount telemetry is getting into the EFD
#Add telemetry as needed
#For the "setCompensateMode" of the Hexapods the "elevation" of the telescope is needed
mtm_sim_Elevation_post_stop_1= await mtm_sim.tel_elevation.aget(timeout=10.)
print("Simulated mount elevation", mtm_sim_Elevation_post_stop_1.angleActual)
print("Simulated mount elevation", mtm_sim_Elevation_post_stop_1.timestamp)
await asyncio.sleep(10)

TimeoutError: 

In [ ]:
#Get timestamp from last elevation telemetry 
mtm_sim_Elevation_post_stop_2= await mtm_sim.tel_elevation.aget(timeout=10.)
print("Simulated mount elevation", mtm_sim_Elevation_post_stop_2.timestamp)

In [ ]:
#Check if the input to the EFD has stopped
diff_time = mtm_sim_Elevation_post_stop_1.timestamp - mtm_sim_Elevation_post_stop_2.timestamp
print (diff_time)

if (mtm_sim_Elevation_post_stop_1.timestamp - mtm_sim_Elevation_post_stop_2.timestamp==0):
    print("Mount has stopped sending telemetry")
else:
    raise Exception("Mount is still sending telemetry!")

In [ ]:
#Stop the script controller
await script.close()

In [9]:
#Start the MTMount controller
mtm_control = salobj.Controller(name="MTMount", index=0)
await mtm_control.start_task

In [11]:
#To check if the input to the EFD has started prepare the EFD readout
#may need to wait a few seconds before event shows up in EFD
#client = EfdClient('ncsa_teststand_efd')
client = EfdClient('summit_efd')

In [14]:
#Send a test command 
mtm_control.evt_target.set_put(elevation=45)

False

In [13]:
#Read from the EFD
end = Time(datetime.now())
start = end - timedelta(hours=3)
logeventTarget = await client.select_time_series('lsst.sal.MTMount.logevent_target', '*', start.tai, end.tai)
print("Mount controller elevation from the EFD:")
logeventTarget.elevation

Mount controller elevation from the EFD:


2021-03-23 17:32:38.692000+00:00    45
Name: elevation, dtype: int64

# Set the MTMount Controller to the needed values and exectute the tests from another note:

In [ ]:
# Set the elevation angle. Do this for every desired elevation angle.
mtm_control.evt_target.set_put(elevation=45)

# After the tests:

In [15]:
#Stop the MTMount controller
await mtm_control.close()

In [ ]:
#Send a test command 
mtm_control.evt_target.set_put(elevation=35)

In [ ]:
#Check that no more telemetry from the MTMount controller is going into the EFD
logeventTarget = await client.select_time_series('lsst.sal.MTMount.logevent_target', '*', start.tai, end.tai)
print("Mount controller elevation from the EFD:")
logeventTarget.elevation

In [ ]:
#Start the script controller
script = salobj.Controller("Script", index=42658885)
mtm_sim = salobj.Remote(script.domain, "MTMount")

In [ ]:
#Start the mount Simluator controller
await mtm_sim.start_task

In [ ]:
#Send the MTMount simulator component into ENABLE again
await salobj.set_summary_state(mtm_sim, salobj.State.ENABLED)

In [ ]:
#Check that the EFD is getting MTMount siumlator telemetry again
#Add telemetry as needed
#For the "setCompensateMode" of the Hexapods the "elevation" of the telescope is needed
mtm_sim_Elevation_post_stop_1= await mtm_sim.tel_elevation.aget(timeout=10.)
print("Simulated mount elevation", mtm_sim_Elevation_post_stop_1.angleActual)
print("Simulated mount elevation", mtm_sim_Elevation_post_stop_1.timestamp)
await asyncio.sleep(10)

In [ ]:
#Get timestamp from last elevation telemetry 
mtm_sim_Elevation_post_stop_2 = await mtm_sim.tel_elevation.aget(timeout=10.)
print("Simulated mount elevation", mtm_sim_Elevation_post_stop_2.timestamp)

In [ ]:
#Check if the input to the EFD has stopped
diff_time = mtm_sim_Elevation_post_stop_1.timestamp - mtm_sim_Elevation_post_stop_2.timestamp
print (diff_time)

if (mtm_sim_Elevation_post_stop_1.timestamp - mtm_sim_Elevation_post_stop_2.timestamp != 0):
    print("Mount is receiving telemetry again")
else:
    raise Exception("Mount is still does not receive telemetry!")